In [1]:
%matplotlib inline

from keras.models import load_model
from matplotlib import pyplot as plt
from matplotlib import pylab
import numpy as np
from sqlite3 import dbapi2 as sqlite
import pickle

Using TensorFlow backend.
/home/jwu/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
con = sqlite.connect('barley_30m.db')
c = con.cursor()

img, = c.execute('select rgb from img limit 1').fetchone()
plt.imshow(pickle.loads(img, encoding='bytes'))
plt.title('In RGB space')

TypeError: a bytes-like object is required, not 'str'

In [ ]:
model = load_model('model.h5')

In [ ]:
model.summary()
model.layers[0]

In [ ]:
top_layer = model.layers[0]
plt.imshow(
    top_layer.get_weights()[0][:, :, :, 0].squeeze(), 
    cmap=plt.get_cmap('gray'),
)

In [ ]:
from vis.visualization import visualize_activation
from vis.utils import utils
from keras import activations

from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = (18, 6)

# Utility to search for layer index by name.
# Alternatively we can specify this as -1 since it corresponds to the last layer.
layer_idx = -1

# Swap softmax with linear
model.layers[layer_idx].activation = activations.linear
model = utils.apply_modifications(model)

# This is the output node we want to maximize.
filter_idx = 0
img = visualize_activation(model, layer_idx, filter_indices=filter_idx)
plt.imshow(img[..., 0])

In [ ]:
for output_idx in np.arange(10):
    # Lets turn off verbose output this time to avoid clutter and just see the output.
    img = visualize_activation(model, layer_idx, filter_indices=output_idx, input_range=(0., 1.))
    plt.figure()
    plt.title('Networks perception of {}'.format(output_idx))
    plt.imshow(img[..., 0])

In [ ]:
len(top_layer.get_weights())
model.layers[0].get_weights()[0][:, :, :, 0].shape

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))
